In [1]:
import zipfile
import os
import json
from pprint import pprint
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import re
import seaborn as sb
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
# First file to start with
df = pd.read_json("C:/Users/Vidyjay/Desktop/FAKE NEWS DETECTION/SourceCode/Scraping data/tempdata/articles/2018-01-01.json" , encoding='utf-8')

In [3]:
# Combining all json files to a one data frame
count = 0
for filename in os.listdir("C:/Users/Vidyajay/Desktop/FAKE NEWS DETECTION/SourceCode/Scraping data/tempdata/articles"):
    count+=1
    if count>1:
        file_path = "C:/Users/Vidyajay/Desktop/FAKE NEWS DETECTION/SourceCode/Scraping data/tempdata/articles/" + filename
        #print(file_path)
        df_ = pd.read_json(file_path , encoding='utf-8')
        df = pd.concat(objs= [df,df_], axis=0,ignore_index=True)
        
print(df.shape)

This is the  100 th file -->
This is the  200 th file -->
(60104, 12)


In [4]:
# Getting bodytext from the dataframe
lst = []
for i in range(df.shape[0]):
    lst.append(df.fields[i]["bodyText"])

In [5]:
# Creating a column with bodytext
df["bodyText"] = lst

In [6]:
# Getting headlines from the dataframe
lst_head = []
for i in range(df.shape[0]):
        lst_head.append(df.fields[i]["headline"])

In [7]:
# Creating a column with headline
df["headline"] = lst_head

In [8]:
# Getting rid of the embty bodies
df = df[df.bodyText != ""]

In [9]:
# Total Number of Articles
df.shape

(59151, 14)

In [10]:
# Filtering articles based on topics
df = df[(df.sectionName == 'US news') | (df.sectionName == 'Business') | (df.sectionName == 'Politics') | (df.sectionName == 'World news')]

In [11]:
# Getting to see the dataframe info (data type, non-null value etc.)
df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13071 entries, 0 to 13070
Data columns (total 14 columns):
apiUrl                13071 non-null object
fields                13071 non-null object
id                    13071 non-null object
isHosted              13071 non-null bool
pillarId              13071 non-null object
pillarName            13071 non-null object
sectionId             13071 non-null object
sectionName           13071 non-null object
type                  13071 non-null object
webPublicationDate    13071 non-null object
webTitle              13071 non-null object
webUrl                13071 non-null object
bodyText              13071 non-null object
headline              13071 non-null object
dtypes: bool(1), object(13)
memory usage: 1.3+ MB


In [12]:
df.describe

<bound method NDFrame.describe of                                                   apiUrl  \
0      https://content.guardianapis.com/world/2018/ja...   
1      https://content.guardianapis.com/world/2018/ja...   
2      https://content.guardianapis.com/world/2018/ja...   
3      https://content.guardianapis.com/world/2018/ja...   
4      https://content.guardianapis.com/politics/2018...   
5      https://content.guardianapis.com/world/2018/ja...   
6      https://content.guardianapis.com/world/2018/ja...   
7      https://content.guardianapis.com/world/2018/ja...   
8      https://content.guardianapis.com/politics/2018...   
9      https://content.guardianapis.com/world/2018/ja...   
10     https://content.guardianapis.com/politics/2017...   
11     https://content.guardianapis.com/us-news/2018/...   
12     https://content.guardianapis.com/us-news/2018/...   
13     https://content.guardianapis.com/world/2018/ja...   
14     https://content.guardianapis.com/world/2018/ja...   
15    

In [13]:
# To see how many articles in each category
from collections import Counter
Counter(df.sectionName)

Counter({'World news': 5353,
         'Politics': 2687,
         'US news': 2422,
         'Business': 2609})

In [14]:
# Overview for the dataframe
df.dropna(subset=['bodyText','headline'],inplace=True)
print(df.shape)
#df.head(100)

(13071, 14)


In [15]:
# Publication data range
df.webPublicationDate.min() ,df.webPublicationDate.max()

('2018-01-01T03:05:43Z', '2018-10-01T23:01:08Z')

In [16]:
# Saving the cleaned data into csv file
df.to_csv(r"C:\Users\Nandini\Desktop\sa\SourceCode\Cleaning Data\guardian cleaned.csv")

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13071 entries, 0 to 13070
Data columns (total 14 columns):
apiUrl                13071 non-null object
fields                13071 non-null object
id                    13071 non-null object
isHosted              13071 non-null bool
pillarId              13071 non-null object
pillarName            13071 non-null object
sectionId             13071 non-null object
sectionName           13071 non-null object
type                  13071 non-null object
webPublicationDate    13071 non-null object
webTitle              13071 non-null object
webUrl                13071 non-null object
bodyText              13071 non-null object
headline              13071 non-null object
dtypes: bool(1), object(13)
memory usage: 1.4+ MB


In [19]:
tdf = TfidfVectorizer(stop_words='english',ngram_range=(1,2) )
vectorizer = tdf.fit(df.bodyText)
transformed_text = vectorizer.transform(df.bodyText)
transformed_title = vectorizer.transform(df.headline)

In [21]:
def get_tfidf_term_scores(feature_names):
    '''Returns dictionary with term names and total tfidf scores for all terms in corpus'''
    term_corpus_dict = {}
    # iterate through term index and term 
    for term_ind, term in enumerate(feature_names):
        term_name = feature_names[term_ind]
        term_corpus_dict[term_name] = np.sum(transformed_title.T[term_ind].toarray())
        
    return term_corpus_dict

# list of features created by tfidf
feature_names = tdf.get_feature_names()
term_corpus_dict = get_tfidf_term_scores(feature_names)

KeyboardInterrupt: 

In [ ]:
def get_sorted_tfidf_scores(term_corpus_dict):
     #Returns sort words from highest score to lowest score
    # sort indices from words wit highest score to lowest score
    sortedIndices = np.argsort( list(term_corpus_dict.values()))[::-1]
    # move words and score out of dicts and into arrays
    termNames = np.array(list(term_corpus_dict.keys()))
    scores = np.array(list(term_corpus_dict.values()))
    # sort words and scores
    termNames = termNames[sortedIndices]
    scores = scores[sortedIndices]
    return termNames, scores


termNames, scores = get_sorted_tfidf_scores(term_corpus_dict)

In [ ]:
def plot_tfidf_scores(scores,termNames, n_words = 18):
    '''Returns one plot for Importance of Top Terms'''

    # Create a figure instance, and the two subplots
    fig = plt.figure(figsize = (14, 18))
    
    override = {'fontsize': 'large'}

    fig.add_subplot(221)   #top left
    n_words = 75
    sb.set()
    sb.barplot(x = scores[:n_words], y = termNames[:n_words]);
    plt.title("TFIDF score ".format(n_words));
    plt.xlabel("TFIDF Score");

In [ ]:
plot_tfidf_scores(scores, termNames,  n_words = 18)